In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
from scipy.stats import ks_1samp, uniform, norm, gaussian_kde
import torch
from torchvision.datasets import MNIST
from torchvision import transforms
from torchvision.models import ResNet18_Weights
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset
from joblib import Parallel, delayed

from tqdm import tqdm

import timm

np.random.seed(42)

matplotlib_inline.backend_inline.set_matplotlib_formats("svg")
plt.style.use("math.mplstyle")

plt.rcParams.update({"axes.labelsize": 14})
plt.rcParams.update({"xtick.labelsize": 14})
plt.rcParams.update({"ytick.labelsize": 14})
plt.rcParams.update({"legend.fontsize": 14})
plt.rcParams.update({"axes.titlesize": 16})

In [ ]:
n = 1000
xi = 400

In [ ]:
from typing import Callable

def compute_changepoint_pvalue(n: int, t: int,
                               left_row: Callable[[int], np.ndarray],
                               right_row: Callable[[int], np.ndarray]) -> float:
    left_pvals = []
    if t >= 0:
        for r in range(0, t + 1):
            row = left_row(r)
            if row is None or len(row) == 0:
                continue
            val = row[-1]
            lt = np.sum(row < val)
            eq = np.sum(row == val)
            p = (lt + np.random.uniform(0.0, 1.0) * eq) / len(row)
            left_pvals.append(p)

    right_pvals = []
    if t + 1 < n:
        for r in range(t + 1, n):
            row = right_row(r)
            if row is None or len(row) == 0:
                continue
            val = row[0]
            lt = np.sum(row < val)
            eq = np.sum(row == val)
            p = (lt + np.random.uniform(0.0, 1.0) * eq) / len(row)
            right_pvals.append(p)

    left_method = "exact" if len(left_pvals) <= 300 else "asymp"
    right_method = "exact" if len(right_pvals) <= 300 else "asymp"
    try:
        p_left = ks_1samp(left_pvals, uniform.cdf, method=left_method)[1] if len(left_pvals) > 0 else 1.0
    except Exception:
        p_left = 1.0
    try:
        p_right = ks_1samp(right_pvals, uniform.cdf, method=right_method)[1] if len(right_pvals) > 0 else 1.0
    except Exception:
        p_right = 1.0

    return 1 - (1 - min(p_left, p_right)) ** 2


def compute_changepoint_pvalue_counts(n: int, t: int,
                                      left_counts: Callable[[int], tuple],
                                      right_counts: Callable[[int], tuple]) -> float:
    left_pvals = []
    if t >= 0:
        for r in range(0, t + 1):
            lt, eq, total = left_counts(r)
            if total <= 0:
                continue
            p = (lt + 0.5 * eq) / total
            left_pvals.append(p)

    right_pvals = []
    if t + 1 < n:
        for r in range(t + 1, n):
            lt, eq, total = right_counts(r)
            if total <= 0:
                continue
            p = (lt + 0.5 * eq) / total
            right_pvals.append(p)

    left_method = "exact" if len(left_pvals) <= 300 else "asymp"
    right_method = "exact" if len(right_pvals) <= 300 else "asymp"
    try:
        p_left = ks_1samp(left_pvals, uniform.cdf, method=left_method)[1] if len(left_pvals) > 0 else 1.0
    except Exception:
        p_left = 1.0
    try:
        p_right = ks_1samp(right_pvals, uniform.cdf, method=right_method)[1] if len(right_pvals) > 0 else 1.0
    except Exception:
        p_right = 1.0

    return 1 - (1 - min(p_left, p_right)) ** 2

In [ ]:
def compute_prefix_suffix_modes(predictions, num_classes):
    preds = np.asarray(predictions, dtype=int)
    n = preds.shape[0]
    onehot = np.zeros((n, num_classes), dtype=int)
    onehot[np.arange(n), preds] = 1
    prefix_counts = np.cumsum(onehot, axis=0)
    prefix_mode = prefix_counts.argmax(axis=1)
    suffix_counts_rev = np.cumsum(onehot[::-1], axis=0)
    suffix_counts = suffix_counts_rev[::-1]
    suffix_mode = suffix_counts.argmax(axis=1)
    return prefix_mode, suffix_mode


def run_classification_simulation(predictions, probabilities, num_classes, n_jobs=-1, verbose=1):
    if isinstance(probabilities, torch.Tensor):
        probs = probabilities.detach().cpu().numpy()
    else:
        probs = np.asarray(probabilities)
    preds = np.asarray(predictions, dtype=int)
    n = len(preds)

    prefix_mode, suffix_mode = compute_prefix_suffix_modes(preds, num_classes)
    eps = 1e-12

    def pvalue_for_t(t: int) -> float:
        ref_right = prefix_mode[t] if t >= 0 else prefix_mode[0]
        ref_left = suffix_mode[t + 1] if (t + 1 < n) else None

        def left_row(r: int) -> np.ndarray:
            if ref_left is None:
                return np.array([1.0])
            idx = slice(0, r + 1)
            baseline_labels = prefix_mode[: r + 1]
            num = probs[idx, baseline_labels]
            den = probs[idx, ref_left] + eps
            return num / den

        def right_row(r: int) -> np.ndarray:
            idx = slice(r, n)
            baseline_labels = suffix_mode[r:]
            num = probs[idx, baseline_labels]
            den = probs[idx, ref_right] + eps
            return num / den

        return compute_changepoint_pvalue(n, t, left_row, right_row)

    p_values = Parallel(n_jobs=n_jobs, verbose=verbose)(delayed(pvalue_for_t)(t) for t in range(n - 1))
    return np.array(p_values)

In [ ]:
def pvalues_from_kappa(left_kappa, right_kappa, n_jobs=-1, verbose=1):
    n_minus_1 = len(left_kappa)
    n = n_minus_1 + 1

    def pvalue_for_t(t: int) -> float:
        def left_row(r: int):
            return left_kappa[t][r]

        def right_row(r: int):
            return right_kappa[t][r]

        return compute_changepoint_pvalue(n, t, left_row, right_row)

    p_values = Parallel(n_jobs=n_jobs, verbose=verbose)(delayed(pvalue_for_t)(t) for t in range(n_minus_1))
    return np.array(p_values)


def compute_kappa_classification(predictions, probabilities, num_classes, eps=1e-12):
    if isinstance(probabilities, torch.Tensor):
        probs = probabilities.detach().cpu().numpy()
    else:
        probs = np.asarray(probabilities)
    preds = np.asarray(predictions, dtype=int)
    n = len(preds)

    prefix_mode, suffix_mode = compute_prefix_suffix_modes(preds, num_classes)

    left_kappa = []
    right_kappa = []

    for t in range(n - 1):
        ref_right = prefix_mode[t]
        ref_left = suffix_mode[t + 1] if (t + 1 < n) else None

        left_rows = {}
        if ref_left is None:
            for r in range(0, t + 1):
                left_rows[r] = np.array([1.0])
        else:
            for r in range(0, t + 1):
                j_idx = np.arange(0, r + 1)
                baseline_labels = prefix_mode[: r + 1]
                num = probs[j_idx, baseline_labels]
                den = probs[j_idx, ref_left] + eps
                left_rows[r] = num / den

        right_rows = {}
        for r in range(t + 1, n):
            j_idx = np.arange(r, n)
            baseline_labels = suffix_mode[r:]
            num = probs[j_idx, baseline_labels]
            den = probs[j_idx, ref_right] + eps
            right_rows[r] = num / den

        left_kappa.append(left_rows)
        right_kappa.append(right_rows)

    return left_kappa, right_kappa


def compute_kappa_oracle(timeseries, f0=None, f1=None):
    if f0 is None:
        f0 = norm(loc=-1, scale=1)
    if f1 is None:
        f1 = norm(loc=1, scale=1)

    x = np.asarray(timeseries).reshape(-1)
    n = len(x)
    eps = 1e-12

    left_scores = (f1.pdf(x) + eps) / (f0.pdf(x) + eps)
    right_scores = (f0.pdf(x) + eps) / (f1.pdf(x) + eps)

    left_kappa = []
    right_kappa = []
    for t in range(n - 1):
        left_rows = {r: left_scores[: r + 1] for r in range(0, t + 1)}
        right_rows = {r: right_scores[r:] for r in range(t + 1, n)}
        left_kappa.append(left_rows)
        right_kappa.append(right_rows)
    return left_kappa, right_kappa


def compute_kappa_parametric(timeseries):
    x = np.asarray(timeseries).reshape(-1)
    n = len(x)
    eps = 1e-12

    cumsum = np.cumsum(x)
    r_idx = np.arange(n)
    mu0 = cumsum / (r_idx + 1)

    rcumsum = np.cumsum(x[::-1])

    def suffix_mean(start: int) -> float:
        length = n - start
        total = rcumsum[n - 1 - start]
        return total / length

    left_kappa = []
    right_kappa = []
    for t in range(n - 1):
        mu1_suffix = suffix_mean(t + 1) if (t + 1 < n) else None
        mu0_t = mu0[t]

        left_rows = {}
        if mu1_suffix is None:
            for r in range(0, t + 1):
                left_rows[r] = np.array([1.0])
        else:
            for r in range(0, t + 1):
                vals = x[: r + 1]
                m0 = mu0[r]
                num = np.exp(-0.5 * (vals - mu1_suffix) ** 2)
                den = np.exp(-0.5 * (vals - m0) ** 2) + eps
                left_rows[r] = num / den

        right_rows = {}
        for r in range(t + 1, n):
            vals = x[r:]
            mu1_r = suffix_mean(r)
            num = np.exp(-0.5 * (vals - mu0_t) ** 2)
            den = np.exp(-0.5 * (vals - mu1_r) ** 2) + eps
            right_rows[r] = num / den

        left_kappa.append(left_rows)
        right_kappa.append(right_rows)

    return left_kappa, right_kappa


def compute_kde(data):
    data = np.asarray(data).reshape(-1)
    if len(data) <= 1:
        mean = data[0] if len(data) == 1 else 0.0
        return lambda x: norm.pdf(x, loc=mean, scale=0.1)
    return gaussian_kde(data, bw_method="scott")


def compute_kappa_kde(timeseries):
    x = np.asarray(timeseries).reshape(-1)
    n = len(x)
    eps = 1e-12

    left_kappa = []
    right_kappa = []

    for t in range(n - 1):
        f1_suffix = compute_kde(x[t + 1 :]) if (t + 1 < n) else None
        f0_t = compute_kde(x[: t + 1]) if t >= 0 else compute_kde(x[:1])

        left_rows = {}
        if f1_suffix is None:
            for r in range(0, t + 1):
                left_rows[r] = np.array([1.0])
        else:
            for r in range(0, t + 1):
                f0_r = compute_kde(x[: r + 1])
                vals = x[: r + 1]
                num = f1_suffix(vals)
                den = f0_r(vals) + eps
                left_rows[r] = num / den

        right_rows = {}
        for r in range(t + 1, n):
            f1_r = compute_kde(x[r:])
            vals = x[r:]
            num = f0_t(vals)
            den = f1_r(vals) + eps
            right_rows[r] = num / den

        left_kappa.append(left_rows)
        right_kappa.append(right_rows)

    return left_kappa, right_kappa

In [ ]:
def pvalues_from_row_functions(n, left_row_fn, right_row_fn, n_jobs=1, verbose=0):
    def pvalue_for_t(t: int) -> float:
        def left_row(r: int):
            return left_row_fn(t, r)
        def right_row(r: int):
            return right_row_fn(t, r)
        return compute_changepoint_pvalue(n, t, left_row, right_row)

    if n_jobs == 1:
        return np.array([pvalue_for_t(t) for t in range(n - 1)])
    else:
        return np.array(Parallel(n_jobs=n_jobs, verbose=verbose)(delayed(pvalue_for_t)(t) for t in range(n - 1)))


def make_classification_kappa_accessors(predictions, probabilities, num_classes, eps=1e-12):
    if isinstance(probabilities, torch.Tensor):
        probs = probabilities.detach().cpu().numpy()
    else:
        probs = np.asarray(probabilities)
    preds = np.asarray(predictions, dtype=int)
    n = len(preds)

    prefix_mode, suffix_mode = compute_prefix_suffix_modes(preds, num_classes)

    def left_row_fn(t: int, r: int) -> np.ndarray:
        if t + 1 >= n:
            return np.array([1.0])
        ref_left = suffix_mode[t + 1]
        j_idx = np.arange(0, r + 1)
        baseline_labels = prefix_mode[: r + 1]
        num = probs[j_idx, baseline_labels]
        den = probs[j_idx, ref_left] + eps
        return num / den

    def right_row_fn(t: int, r: int) -> np.ndarray:
        ref_right = prefix_mode[t]
        j_idx = np.arange(r, n)
        baseline_labels = suffix_mode[r:]
        num = probs[j_idx, baseline_labels]
        den = probs[j_idx, ref_right] + eps
        return num / den

    return left_row_fn, right_row_fn


def make_oracle_row_functions(timeseries):
    x = np.asarray(timeseries).reshape(-1)
    n = len(x)
    f0 = norm(loc=-1, scale=1)
    f1 = norm(loc=1, scale=1)
    eps = 1e-12
    left_scores = (f1.pdf(x) + eps) / (f0.pdf(x) + eps)
    right_scores = (f0.pdf(x) + eps) / (f1.pdf(x) + eps)

    def left_row_fn(t: int, r: int) -> np.ndarray:
        return left_scores[: r + 1]

    def right_row_fn(t: int, r: int) -> np.ndarray:
        return right_scores[r:]

    return left_row_fn, right_row_fn


def make_parametric_row_functions(timeseries):
    x = np.asarray(timeseries).reshape(-1)
    n = len(x)
    eps = 1e-12

    cumsum = np.cumsum(x)
    r_idx = np.arange(n)
    mu0 = cumsum / (r_idx + 1)

    rcumsum = np.cumsum(x[::-1])
    def suffix_mean(start: int) -> float:
        length = n - start
        total = rcumsum[n - 1 - start]
        return total / length

    def left_row_fn(t: int, r: int) -> np.ndarray:
        if t + 1 >= n:
            return np.array([1.0])
        mu1_suffix = suffix_mean(t + 1)
        vals = x[: r + 1]
        m0 = mu0[r]
        num = np.exp(-0.5 * (vals - mu1_suffix) ** 2)
        den = np.exp(-0.5 * (vals - m0) ** 2) + eps
        return num / den

    def right_row_fn(t: int, r: int) -> np.ndarray:
        mu0_t = mu0[t]
        vals = x[r:]
        mu1_r = suffix_mean(r)
        num = np.exp(-0.5 * (vals - mu0_t) ** 2)
        den = np.exp(-0.5 * (vals - mu1_r) ** 2) + eps
        return num / den

    return left_row_fn, right_row_fn


def make_kde_row_functions(timeseries):
    x = np.asarray(timeseries).reshape(-1)
    n = len(x)
    eps = 1e-12

    prefix_kdes = []
    for r in range(n):
        prefix_kdes.append(compute_kde(x[: r + 1]))
    suffix_kdes = []
    for r in range(n):
        suffix_kdes.append(compute_kde(x[r:]))

    def left_row_fn(t: int, r: int) -> np.ndarray:
        if t + 1 >= n:
            return np.array([1.0])
        f1_suffix = suffix_kdes[t + 1]
        f0_r = prefix_kdes[r]
        vals = x[: r + 1]
        num = f1_suffix(vals)
        den = f0_r(vals) + eps
        return num / den

    def right_row_fn(t: int, r: int) -> np.ndarray:
        f0_t = prefix_kdes[t]
        f1_r = suffix_kdes[r]
        vals = x[r:]
        num = f0_t(vals)
        den = f1_r(vals) + eps
        return num / den

    return left_row_fn, right_row_fn

# Gaussian mean change

In [ ]:
timeseries = np.concatenate([np.random.normal(-1, 1, xi), np.random.normal(1, 1, n - xi)])

time_indices = np.arange(1, n + 1)

plt.plot(time_indices, timeseries)
plt.show()

## Oracle likelihood ratio score

### Two-stage kappa pipeline
For each candidate t and each r, we recompute the required scores for all j in the row to form `kappa_{rj}^{(t)}`, then compute ranks and KS p-values via a generic wrapper. This matches the MNIST reference algorithm and is applied to all simulations.

In [ ]:
def run_single_simulation_oracle(n, xi, timeseries):
    f_0 = norm(-1, 1)
    f_1 = norm(1, 1)

    left_scores = f_1.pdf(timeseries) / f_0.pdf(timeseries)
    right_scores = f_0.pdf(timeseries) / f_1.pdf(timeseries)

    def pvalue_for_t(t: int) -> float:
        def left_row(r: int) -> np.ndarray:
            return left_scores[: r + 1]

        def right_row(r: int) -> np.ndarray:
            return right_scores[r:]

        return compute_changepoint_pvalue(n, t, left_row, right_row)

    p_values = Parallel(n_jobs=-1, verbose=1)(delayed(pvalue_for_t)(t) for t in range(n - 1))
    return np.array(p_values)

p_values = run_single_simulation_oracle(n, xi, timeseries)

In [ ]:
def plot_and_report(p_values, xi, title, outpath):
    time_indices_p = np.arange(1, len(p_values) + 1)
    plt.plot(time_indices_p, p_values)
    plt.axvline(x=xi, color="red", linestyle="--", label="Changepoint ($\\xi = 400$)")
    plt.axhline(0.05, color="green", linestyle=":", label="Threshold ($\\alpha = 0.05$)")
    plt.xlabel("$t$")
    plt.title(title)
    plt.legend()
    plt.savefig(outpath)
    plt.show()

    detected_changepoint = np.argmax(p_values) + 1
    print(f"\nTrue change point: {xi}")
    print(f"Detected change point: {detected_changepoint}")
    print(f"Detection error: {abs(detected_changepoint - xi)}")
    print(f"Size of confidence set: {np.sum(p_values > 0.05)}")
    print(f"Changepoint in confidence set: {p_values[xi-1] > 0.05}")
    print(f"CI: {np.where(p_values > 0.05)[0] + 1}")

plot_and_report(p_values, xi, "p-values for Gaussian mean change (oracle score)", "images/oracle.pdf")

## Parametric learned score

In [ ]:
def run_single_simulation_parametric(n, xi, timeseries):
    left_row_fn, right_row_fn = make_parametric_row_functions(timeseries)
    return pvalues_from_row_functions(len(timeseries), left_row_fn, right_row_fn, n_jobs=-1, verbose=1)

p_values = run_single_simulation_parametric(n, xi, timeseries)

In [ ]:
plot_and_report(p_values, xi, "p-values for Gaussian mean change (parametric learned score)", "images/parametric.pdf")

## Kernel density estimator (KDE) learned score

In [ ]:
def run_single_simulation_kde(n, xi, timeseries, n_jobs=-1, verbose=1):
    left_row_fn, right_row_fn = make_kde_row_functions(timeseries)
    return pvalues_from_row_functions(len(timeseries), left_row_fn, right_row_fn, n_jobs=n_jobs, verbose=verbose)

p_values = run_single_simulation_kde(n, xi, timeseries)

In [ ]:
plot_and_report(p_values, xi, "p-values for Gaussian mean change (KDE learned score)", "images/kde.pdf")

## Coverage and width simulations

In [ ]:
def _oracle_p_value_for_t(t, timeseries):
    n = len(timeseries)
    f_0 = norm(-1, 1)
    f_1 = norm(1, 1)
    left_scores = f_1.pdf(timeseries) / f_0.pdf(timeseries)
    right_scores = f_0.pdf(timeseries) / f_1.pdf(timeseries)

    def left_row(r: int) -> np.ndarray:
        return left_scores[: r + 1]

    def right_row(r: int) -> np.ndarray:
        return right_scores[r:]

    return compute_changepoint_pvalue(n, t, left_row, right_row)


def run_single_simulation(n, xi):
    timeseries = np.concatenate([np.random.normal(-1, 1, xi), np.random.normal(1, 1, n - xi)])
    p_values = Parallel(n_jobs=-1, verbose=1)(delayed(_oracle_p_value_for_t)(t, timeseries) for t in range(n - 1))
    return np.array(p_values)


def run_simulation_study(n_simulations=1000, n=500, xi=200):
    all_p_values = np.zeros((n_simulations, n - 1))
    coverages_95 = []
    coverages_50 = []
    widths_95 = []
    widths_50 = []
    detected_cps = []
    pbar = tqdm(range(n_simulations))
    for i in pbar:
        p_values = run_single_simulation(n, xi)
        all_p_values[i] = p_values
        coverage_95 = p_values[xi - 1] > 0.05
        coverage_50 = p_values[xi - 1] > 0.50
        width_95 = np.sum(p_values > 0.05)
        width_50 = np.sum(p_values > 0.50)
        detected_cp = np.argmax(p_values) + 1
        coverages_95.append(coverage_95)
        coverages_50.append(coverage_50)
        widths_95.append(width_95)
        widths_50.append(width_50)
        detected_cps.append(detected_cp)
        if i > 0:
            running_cov_95 = np.mean(coverages_95)
            running_cov_50 = np.mean(coverages_50)
            running_width_95 = np.mean(widths_95)
            running_error = np.mean([abs(cp - xi) for cp in detected_cps])
            pbar.set_description(f"Cov95:{running_cov_95:.3f} Cov50:{running_cov_50:.3f} W95:{running_width_95:.1f} Err:{running_error:.1f}")
    return all_p_values

In [ ]:
all_p_values = run_simulation_study(n_simulations=1000, n=500, xi=200)

In [ ]:
xi = 200
n_simulations, n_minus_1 = all_p_values.shape

coverage_95 = all_p_values[:, xi-1] > 0.05
coverage_50 = all_p_values[:, xi-1] > 0.50

widths_95 = np.sum(all_p_values > 0.05, axis=1)
widths_50 = np.sum(all_p_values > 0.50, axis=1)

detected_cps = np.argmax(all_p_values, axis=1) + 1
detection_errors = np.abs(detected_cps - xi)

print(f"95% Coverage: {np.mean(coverage_95):.3f}")
print(f"50% Coverage: {np.mean(coverage_50):.3f}")
print(f"Average Width (95%): {np.mean(widths_95):.1f}")
print(f"Average Width (50%): {np.mean(widths_50):.1f}")
print(f"Average Detection Error: {np.mean(detection_errors):.1f}")

In [ ]:
np.save('all_p_values.npy', all_p_values)

# MNIST digit change

In [ ]:
def get_mnist_trained_model(device="cpu"):
    class MNISTModel(torch.nn.Module):
        def __init__(self):
            super(MNISTModel, self).__init__()
            self.conv1 = torch.nn.Conv2d(1, 32, 3, 1)
            self.conv2 = torch.nn.Conv2d(32, 64, 3, 1)
            self.dropout1 = torch.nn.Dropout(0.25)
            self.dropout2 = torch.nn.Dropout(0.5)
            self.fc1 = torch.nn.Linear(9216, 128)
            self.fc2 = torch.nn.Linear(128, 10)

        def forward(self, x):
            x = self.conv1(x)
            x = torch.nn.functional.relu(x)
            x = self.conv2(x)
            x = torch.nn.functional.relu(x)
            x = torch.nn.functional.max_pool2d(x, 2)
            x = self.dropout1(x)
            x = torch.flatten(x, 1)
            x = self.fc1(x)
            x = torch.nn.functional.relu(x)
            x = self.dropout2(x)
            x = self.fc2(x)
            return x

    model = MNISTModel().to(device)

    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    )

    train_dataset = MNIST(root="./data", train=True, download=True, transform=transform)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64)

    optimizer = torch.optim.Adam(model.parameters())
    criterion = torch.nn.CrossEntropyLoss()

    print("Training MNIST model...")
    model.train()
    for epoch in range(1):
        for batch_idx, (data, target) in enumerate(tqdm(train_loader)):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                print(
                    f"Epoch: {epoch} [{batch_idx*len(data)}/{len(train_loader.dataset)} "
                    f"({100. * batch_idx / len(train_loader):.0f}%)]\\tLoss: {loss.item():.6f}"
                )

    model.eval()
    return model


def predict_digit(model, image, device="cpu"):
    image = image.reshape(1, 1, 28, 28)
    image_tensor = torch.tensor(image, device=device)

    with torch.no_grad():
        outputs = torch.softmax(model(image_tensor), dim=1).cpu()
        predicted = outputs.argmax(dim=1).item()
    return (predicted, outputs)


def generate_mnist_dataset(length, changepoint, digit1=3, digit2=7):
    transform = transforms.ToTensor()
    mnist_data = MNIST(root="./data", train=True, download=True, transform=transform)
    data = mnist_data.data.numpy()
    targets = mnist_data.targets.numpy()

    images_digit1 = data[targets == digit1]
    images_digit2 = data[targets == digit2]
    np.random.shuffle(images_digit1)
    np.random.shuffle(images_digit2)

    n1 = changepoint + 1
    n2 = length - n1
    if n1 > len(images_digit1) or n2 > len(images_digit2):
        raise ValueError("Insufficient images for the specified digits and length.")

    data1 = images_digit1[:n1]
    data2 = images_digit2[:n2]
    x = np.concatenate([data1, data2], axis=0)

    x = x.reshape(length, -1).astype(np.float32) / 255.0
    return x

In [ ]:
def compute_sequential_scores(x, model, device="cpu", batch_size=128):
    model.to(device)
    model.eval()
    n = len(x)
    preds = []
    probs_list = []
    x_tensor = torch.tensor(x, dtype=torch.float32, device=device).view(n, 1, 28, 28)
    x_tensor = (x_tensor - 0.1307) / 0.3081
    with torch.no_grad():
        for start in tqdm(range(0, n, batch_size)):
            end = min(start + batch_size, n)
            batch = x_tensor[start:end]
            logits = model(batch)
            batch_probs = torch.softmax(logits, dim=1).detach().cpu()
            probs_list.append(batch_probs)
            preds.extend(batch_probs.argmax(dim=1).tolist())
    probabilities = torch.cat(probs_list, dim=0)
    return preds, probabilities

In [ ]:
model = get_mnist_trained_model()

In [ ]:
xi = 400

def _mnist_p_values(predictions, probabilities):
    num_classes = 10
    left_row_fn, right_row_fn = make_classification_kappa_accessors(predictions, probabilities, num_classes)
    n = len(predictions)
    return pvalues_from_row_functions(n, left_row_fn, right_row_fn, n_jobs=-1, verbose=1)


def run_mnist_simulation(n, xi):
    x = generate_mnist_dataset(n, xi)
    predictions, probabilities = compute_sequential_scores(x, model)
    return _mnist_p_values(predictions, probabilities)

p_values = run_mnist_simulation(n, xi)

In [ ]:
plot_and_report(p_values, xi, "p-values for MNIST digit change (digit classifier)", "images/mnist-pvalues.pdf")

# SST-2 sentiment change (LLM)

### Note: Two-stage pipeline
We now compute kappa values `kappa_{rj}^{(t)}` first (left and right) for each simulation, then apply the generic p-value wrapper. This ensures scores are recomputed within each `r` row as required.

In [ ]:
def get_pretrained_sentiment_model(device="cpu"):
    model_name = "distilbert-base-uncased-finetuned-sst-2-english"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    model.to(device)
    model.eval()
    return model, tokenizer


def generate_sentiment_dataset(length, changepoint, dataset_name="sst2"):
    dataset = load_dataset(dataset_name)

    train_data = dataset["train"]
    positive_texts = [item["sentence"] for item in train_data if item["label"] == 1]
    negative_texts = [item["sentence"] for item in train_data if item["label"] == 0]

    random.shuffle(positive_texts)
    random.shuffle(negative_texts)

    n1 = changepoint + 1
    n2 = length - n1

    if n1 > len(positive_texts) or n2 > len(negative_texts):
        raise ValueError("Insufficient texts for the specified length and changepoint.")

    texts_before = positive_texts[:n1]
    texts_after = negative_texts[:n2]

    texts = texts_before + texts_after
    true_labels = [1] * n1 + [0] * n2

    return texts, true_labels


def generate_mixed_sentiment_dataset(length, changepoint, dataset_name="sst2"):
    dataset = load_dataset(dataset_name)

    train_data = dataset["train"]
    positive_texts = [item["sentence"] for item in train_data if item["label"] == 1]
    negative_texts = [item["sentence"] for item in train_data if item["label"] == 0]

    random.shuffle(positive_texts)
    random.shuffle(negative_texts)

    n_pre = changepoint + 1
    n_post = length - n_pre

    n_pos_pre = int(n_pre * 0.6)
    n_neg_pre = n_pre - n_pos_pre

    n_pos_post = int(n_post * 0.4)
    n_neg_post = n_post - n_pos_post

    if n_pos_pre + n_pos_post > len(positive_texts) or n_neg_pre + n_neg_post > len(
        negative_texts
    ):
        raise ValueError(
            "Insufficient texts for the specified distribution and length."
        )

    pre_pos_texts = positive_texts[:n_pos_pre]
    pre_neg_texts = negative_texts[:n_neg_pre]
    pre_texts = pre_pos_texts + pre_neg_texts
    pre_labels = [1] * n_pos_pre + [0] * n_neg_pre

    pre_combined = list(zip(pre_texts, pre_labels))
    random.shuffle(pre_combined)
    pre_texts, pre_labels = zip(*pre_combined)

    post_pos_texts = positive_texts[n_pos_pre : n_pos_pre + n_pos_post]
    post_neg_texts = negative_texts[n_neg_pre : n_neg_pre + n_neg_post]
    post_texts = post_pos_texts + post_neg_texts
    post_labels = [1] * n_pos_post + [0] * n_neg_post

    post_combined = list(zip(post_texts, post_labels))
    random.shuffle(post_combined)
    post_texts, post_labels = zip(*post_combined)

    texts = list(pre_texts) + list(post_texts)
    true_labels = list(pre_labels) + list(post_labels)

    return texts, true_labels


def predict_sentiment(model, tokenizer, text, device="cpu"):
    inputs = tokenizer(
        text, return_tensors="pt", padding=True, truncation=True, max_length=512
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1).cpu()
        predicted = probs.argmax(dim=1).item()

    return (predicted, probs.squeeze())

## Plot examples

In [ ]:
def compute_sequential_sentiment_scores(texts, device="cpu", batch_size=16, max_length=128):
    model_name = "distilbert-base-uncased-finetuned-sst-2-english"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    model.to(device)
    model.eval()

    n = len(texts)
    preds = []
    probs_list = []
    with torch.no_grad():
        for start in tqdm(range(0, n, batch_size)):
            end = min(start + batch_size, n)
            batch_texts = texts[start:end]
            inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=max_length).to(device)
            outputs = model(**inputs)
            batch_probs = torch.softmax(outputs.logits, dim=1).detach().cpu()
            probs_list.append(batch_probs)
            preds.extend(batch_probs.argmax(dim=1).tolist())
    probabilities = torch.cat(probs_list, dim=0)
    return preds, probabilities

## Full sentiment change (pos. to neg.)

In [ ]:
def _sentiment_p_values(predictions, probabilities):
    num_classes = 2
    left_row_fn, right_row_fn = make_classification_kappa_accessors(predictions, probabilities, num_classes)
    n_local = len(predictions)
    return pvalues_from_row_functions(n_local, left_row_fn, right_row_fn, n_jobs=-1, verbose=1)


def run_sentiment_simulation(n, xi, device="cpu"):
    texts, true_labels = generate_sentiment_dataset(n, xi)
    predictions, probabilities = compute_sequential_sentiment_scores(texts, device=device, batch_size=16, max_length=128)
    return _sentiment_p_values(predictions, probabilities)

p_values = run_sentiment_simulation(n, xi)

In [ ]:
plot_and_report(p_values, xi, "p-values for SST-2 sentiment change", "images/sentiment-pvalues.pdf")

## Mixed sentiment change (60% pos. to 60% neg.)

In [ ]:
def run_mixed_sentiment_simulation(n, xi, device="cpu"):
    texts, true_labels = generate_mixed_sentiment_dataset(n, xi)
    predictions, probabilities = compute_sequential_sentiment_scores(texts, device=device, batch_size=16, max_length=128)
    return _sentiment_p_values(predictions, probabilities)

p_values = run_mixed_sentiment_simulation(n, xi)

In [ ]:
plot_and_report(p_values, xi, "p-values for SST-2 mixed sentiment change", "images/sentiment-pvalues-mixed.pdf")

# Human Activity Dataset (HAD) change

In [ ]:
def load_had_dataset():
    import urllib.request
    import zipfile
    import os
    
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip"
    
    os.makedirs("./data", exist_ok=True)
    
    if not os.path.exists("./data/UCI_HAR_Dataset.zip"):
        print("Downloading Human Activity Dataset...")
        urllib.request.urlretrieve(url, "./data/UCI_HAR_Dataset.zip")
    
    if not os.path.exists("./data/UCI HAR Dataset"):
        print("Extracting dataset...")
        with zipfile.ZipFile("./data/UCI_HAR_Dataset.zip", 'r') as zip_ref:
            zip_ref.extractall("./data")
    
    train_file = "./data/UCI HAR Dataset/train/X_train.txt"
    train_labels_file = "./data/UCI HAR Dataset/train/y_train.txt"
    
    if os.path.exists(train_file):
        X_train = np.loadtxt(train_file)
        y_train = np.loadtxt(train_labels_file)
        
        walking_indices = np.where(y_train == 1)[0]
        sitting_indices = np.where(y_train == 4)[0]
        
        n_walking = min(400, len(walking_indices))
        n_sitting = min(600, len(sitting_indices))
        
        if n_walking > 0 and n_sitting > 0:
            selected_walking = walking_indices[100:100+n_walking]
            selected_sitting = sitting_indices[100:100+n_sitting]
            combined_indices = np.concatenate([selected_walking, selected_sitting])
        
            timeseries = X_train[combined_indices, 0]
            true_changepoint = n_walking
            
            return timeseries, true_changepoint


print("Loading Human Activity Dataset...")
timeseries, true_changepoint = load_had_dataset()

n_had = len(timeseries)
xi_had = true_changepoint

print(f"HAD Dataset loaded:")
print(f"Total length: {n_had} samples")
print(f"True changepoint: {xi_had} (activity transition)")

time_indices = np.arange(1, n_had + 1)
plt.plot(time_indices, timeseries)
plt.axvline(x=xi_had + 1, color="red", linestyle="--", label=f"Changepoint ($\\xi = {xi_had}$)")
plt.xlabel("$t$")
plt.ylabel("Accelerometer reading")
plt.title("Human Activity Recognition accelerometer data")
plt.legend()
plt.savefig("images/had-examples.pdf")
plt.show()

pre_change = timeseries[:xi_had]
post_change = timeseries[xi_had:]

print(f"\nDataset Statistics:")
print(f"Pre-change activity mean: {np.mean(pre_change):.3f}, std: {np.std(pre_change):.3f}")
print(f"Post-change activity mean: {np.mean(post_change):.3f}, std: {np.std(post_change):.3f}")
print(f"Mean difference: {np.mean(pre_change) - np.mean(post_change):.3f}")

In [ ]:
def run_had_simulation_kde():
    left_row_fn, right_row_fn = make_kde_row_functions(timeseries)
    return pvalues_from_row_functions(len(timeseries), left_row_fn, right_row_fn, n_jobs=-1, verbose=1)

p_values = run_had_simulation_kde()

In [ ]:
time_indices_p = np.arange(1, n_had)
plot_and_report(p_values, xi_had + 1, "p-values for HAR activity change", "images/had-pvalues.pdf")

# CIFAR-100 class change

In [ ]:
def get_cifar100_pretrained_model(device="cpu"):
    import detectors
    import timm
    print("Loading pretrained ResNet18 model from timm and adapting for CIFAR-100...")
    
    model = timm.create_model("resnet18_cifar100", pretrained=True, num_classes=100)
    model.to(device)
    model.eval()
    return model

In [ ]:
def predict_cifar100_class(model, image, device="cpu"):
    if len(image.shape) == 3:
        image = image.unsqueeze(0)
    elif len(image.shape) == 4 and image.shape[0] != 1:
        image = image[:1]
    
    image_tensor = image.to(device)
    
    with torch.no_grad():
        outputs = torch.softmax(model(image_tensor), dim=1).cpu()
        predicted = outputs.argmax(dim=1).item()
    
    return (predicted, outputs.squeeze())


def generate_cifar100_dataset(length, changepoint, class1=15, class2=47):
    from torchvision.datasets import CIFAR100
    
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761])
    ])
    
    train_data = CIFAR100(root="./data", train=True, download=True, transform=transform)
    test_data = CIFAR100(root="./data", train=False, download=True, transform=transform)
    
    all_images = []
    all_labels = []
    
    for i in range(len(train_data)):
        image, label = train_data[i]
        all_images.append(image)
        all_labels.append(label)
    
    for i in range(len(test_data)):
        image, label = test_data[i]
        all_images.append(image)
        all_labels.append(label)
    
    class1_indices = [i for i, label in enumerate(all_labels) if label == class1]
    class2_indices = [i for i, label in enumerate(all_labels) if label == class2]
    
    np.random.shuffle(class1_indices)
    np.random.shuffle(class2_indices)
    
    n1 = changepoint + 1
    n2 = length - n1
    
    selected_indices = class1_indices[:n1] + class2_indices[:n2]
    selected_images = [all_images[i] for i in selected_indices]
    
    return torch.stack(selected_images)

In [ ]:
def compute_sequential_cifar100_scores(x, model, device="cpu", batch_size=64):
    model.to(device)
    model.eval()
    n = len(x)
    preds = []
    probs_list = []
    with torch.no_grad():
        for start in tqdm(range(0, n, batch_size)):
            end = min(start + batch_size, n)
            batch = x[start:end].to(device)
            logits = model(batch)
            batch_probs = torch.softmax(logits, dim=1).detach().cpu()
            probs_list.append(batch_probs)
            preds.extend(batch_probs.argmax(dim=1).tolist())
    probabilities = torch.cat(probs_list, dim=0)
    return preds, probabilities

In [ ]:
from torchvision.datasets import CIFAR100

cifar100_dataset = CIFAR100(root="./data", train=False, download=True)
class_names = cifar100_dataset.classes

print("CIFAR-100 class names:")
print(f"Class 3: {class_names[3]}")
print(f"Class 4: {class_names[4]}")

transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761]
        ),
    ]
)

train_data = CIFAR100(root="./data", train=True, download=True, transform=transform)
test_data = CIFAR100(root="./data", train=False, download=True, transform=transform)

class3_count_train = sum(1 for _, label in train_data if label == 3)
class4_count_train = sum(1 for _, label in train_data if label == 4)
class3_count_test = sum(1 for _, label in test_data if label == 3)
class4_count_test = sum(1 for _, label in test_data if label == 4)

print(
    f"\nClass 3 ({class_names[3]}) - Train: {class3_count_train}, Test: {class4_count_test}, Total: {class3_count_train + class3_count_test}"
)
print(
    f"Class 4 ({class_names[4]}) - Train: {class4_count_train}, Test: {class4_count_test}, Total: {class4_count_train + class4_count_test}"
)


def generate_cifar100_dataset_fixed(length, changepoint, class1=3, class2=4):
    from torchvision.datasets import CIFAR100

    transform = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761]
            ),
        ]
    )

    train_data = CIFAR100(root="./data", train=True, download=True, transform=transform)
    test_data = CIFAR100(root="./data", train=False, download=True, transform=transform)

    class1_images = []
    class1_labels = []

    for _, (image, label) in enumerate(train_data):
        if label == class1:
            class1_images.append(image)
            class1_labels.append(label)

    for _, (image, label) in enumerate(test_data):
        if label == class1:
            class1_images.append(image)
            class1_labels.append(label)

    class2_images = []
    class2_labels = []

    for _, (image, label) in enumerate(train_data):
        if label == class2:
            class2_images.append(image)
            class2_labels.append(label)

    for _, (image, label) in enumerate(test_data):
        if label == class2:
            class2_images.append(image)
            class2_labels.append(label)

    np.random.shuffle(class1_images)
    np.random.shuffle(class2_images)

    n1 = changepoint + 1
    n2 = length - n1

    selected_class1 = class1_images[:n1]
    selected_class2 = class2_images[:n2]

    all_images = selected_class1 + selected_class2
    all_labels = [class1] * n1 + [class2] * n2

    print(f"Generated dataset: {len(all_images)} images")
    print(f"First {n1} images are class {class1} ({class_names[class1]})")
    print(f"Last {n2} images are class {class2} ({class_names[class2]})")

    actual_labels_start = all_labels[:5]
    actual_labels_end = all_labels[-5:]
    print(f"First 5 labels: {actual_labels_start}")
    print(f"Last 5 labels: {actual_labels_end}")

    return torch.stack(all_images), all_labels


x_fixed, labels_fixed = generate_cifar100_dataset_fixed(n, xi, class1=3, class2=4)
print(f"\nSuccessfully generated dataset with shape: {x_fixed.shape}")

def show_cifar100_examples_with_labels(x, labels, changepoint, n_examples=3):
    mean = torch.tensor([0.5071, 0.4867, 0.4408])
    std = torch.tensor([0.2675, 0.2565, 0.2761])

    def denormalize_image(tensor):
        denorm = tensor * std[:, None, None] + mean[:, None, None]
        return torch.clamp(denorm, 0, 1)

    fig, axes = plt.subplots(1, 5, figsize=(15, 3))

    class_names = CIFAR100(root="./data", train=False, download=True).classes

    time_points = [398, 399, 400]
    titles_before = [r"$t = 398$", r"$t = 399$", r"$t = \xi = 400$"]

    for i, (t, title) in enumerate(zip(time_points, titles_before)):
        idx = t - 1
        img_denorm = denormalize_image(x[idx])
        actual_class = labels[idx]
        class_name = class_names[actual_class]

        axes[i].imshow(img_denorm.permute(1, 2, 0).numpy())
        axes[i].set_title(f"{title}\n{class_name}")
        axes[i].axis("off")

    post_change_points = [401, 402]
    titles_after = [r"$t = 401$", r"$t = 402$"]

    for i, (t, title) in enumerate(zip(post_change_points, titles_after)):
        idx = t - 1
        img_denorm = denormalize_image(x[idx])
        actual_class = labels[idx]
        class_name = class_names[actual_class]

        axes[i + 3].imshow(img_denorm.permute(1, 2, 0).numpy())
        axes[i + 3].set_title(f"{title}\n{class_name}")
        axes[i + 3].axis("off")

    plt.tight_layout()
    plt.savefig("images/cifar100-examples.pdf")
    plt.show()

show_cifar100_examples_with_labels(x_fixed, labels_fixed, xi)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

cifar100_model = get_cifar100_pretrained_model(device)

In [ ]:
n_cifar = 800
xi_cifar = 300

print(f"CIFAR-100 simulation parameters:")
print(f"Total length: {n_cifar}")
print(f"Changepoint: {xi_cifar}")
print(f"Pre-change images needed: {xi_cifar + 1}")
print(f"Post-change images needed: {n_cifar - xi_cifar - 1}")
print(f"Available per class: ~600 (500 train + 100 test)")

In [ ]:
def _cifar100_p_values(predictions, probabilities):
    num_classes = 100
    left_row_fn, right_row_fn = make_classification_kappa_accessors(predictions, probabilities, num_classes)
    n_local = len(predictions)
    return pvalues_from_row_functions(n_local, left_row_fn, right_row_fn, n_jobs=-1, verbose=1)


def run_cifar100_simulation(n, xi, class1=15, class2=47):
    x = generate_cifar100_dataset(n, xi, class1, class2)
    predictions, probabilities = compute_sequential_cifar100_scores(x, cifar100_model, device)
    return _cifar100_p_values(predictions, probabilities)

p_values = run_cifar100_simulation(n_cifar, xi_cifar, class1=3, class2=4)

In [ ]:
plot_and_report(p_values, xi_cifar, "p-values for CIFAR-100 class change (pretrained model)", "images/cifar100-pvalues.pdf")